In [ ]:
# init
import sys
print (sys.version)
import numpy as np
from sympy import *
from IPython.display import display, Markdown, Latex

u, v, t, w = symbols('u v t w', real=True)
#u, v, t = symbols('u v t')
x, y, z = symbols('x y z')
a, b, c = symbols('a b c')

from geo3py import *

%matplotlib inline
#%matplotlib widget
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from spb import plot3d_parametric_surface, plot3d_parametric_line, PB, MB, KB
from spb import plot
from spb import plot3d

import scipy.integrate

# Differentialgeometri og Parametrisk design - 01237 - E22

Kaare. G. S. Hansen, s214282 - DTU

Dato: 16. december 2022

## Opgave 1

![](2022-12-16-15-01-00.png)

In [ ]:
tet1 = Tetraeder([0,0,0], [1,0,0],[0,1,0],[0,0,1])
tet2 = Tetraeder([0,0,0], [1,1,0],[1,0,1],[0,1,1])

display(
    tet1.getMatrix(),
    tet2.getMatrix()
)

![](2022-12-16-15-01-26.png)

In [ ]:
tet2.getMatrix().det()

Da determinanten er $< 0$, er orienteringen negativ

![](2022-12-16-15-01-38.png)

Givet ved numeriske værdi af determinanten over 6.

In [ ]:
EqPrint('Vol(tet_2)', tet2.getVol())

![](2022-12-16-15-01-49.png)

Vi skal løse ligningen $K \cdot K_1 = K_2$. Finder den inverse af $K_1$.

In [ ]:
K = tet2.getMatrix() * tet1.getMatrix().inv()
EqPrint('K', K)

Vi tjekker at $K$ er rigtig ved at se om $K tet_1 = tet_2$

In [ ]:
K * tet1.getMatrix()

Hvilket den heldigvis er.

![](2022-12-16-15-02-06.png)

In [ ]:
result = SVD3(K)
display(
    EqPrint('F', result.F_hat),
    EqPrint('\Sigma', result.S), 
    EqPrint('U', result.U),
    EqPrint('V', result.V),
)

Med Sympys egen SVD, dog ikke på samme form:

In [ ]:
U, S, V = K.singular_value_decomposition()
display(
    EqPrint("U", U),
    EqPrint("\Sigma", S),
    EqPrint("V^T", V.T),
    EqPrint("U \cdot \Sigma \cdot V^T", U*S*V.T),
)

## Opgave 2
![](2022-12-16-15-02-30.png)

In [ ]:
p2 = Matrix([cos(t), sin(t), sqrt(3)*sin(t)])
t_dom = (t, -pi, pi)
curve2 = Curve3D(p2, t_dom)
v2 = curve2.p
EqPrint('p(t)', v2)

In [ ]:
curve2.quickPlot()

![](2022-12-16-15-02-45.png)

In [ ]:
v2 = get_v(curve2.p, curve2.t_dom[0])
EqPrint('v(t)', v2)

Vi har taget farten af kurven, og det ses at udtrykket er forskelligt fra 1. Derfor er den ikke enhedsfart-parametriseret.

In [ ]:
display(
    v2.subs({t:pi/2}),
    v2.subs({t:-pi/2})
)

Det kan dog bemærkes at netop når $t=\frac{\pi}{2}$ eller $t=-\frac{\pi}{2}$, er farten 1.

Dette skal dog gælde for samtlige værdier i intervallet før den er enheds-parametriseret.

![](2022-12-16-15-02-57.png)

In [ ]:
#curve2.getFrenetSerret()
result = get_FrenetSerret(curve2.p, curve2.t_dom[0])
display(
    EqPrint('e(t)', result.e),
    EqPrint('f(t)', result.f),
    EqPrint('g(t)', result.g)
)

![](2022-12-16-15-03-08.png)

In [ ]:
#get_kappa(curve2.p, curve2.t_dom[0], v2)
EqPrint('\kappa(t)', curve2.getKappa())

![](2022-12-16-15-03-23.png)

In [ ]:
EqPrint('\\tau(t)', curve2.getTau())

Bemærker en torsion på 0 for alle værdier af $t$

![](2022-12-16-15-03-41.png)

In [ ]:
r2 = u*p2
u_dom = (u, Rational(1,2), 1)
surf2 = Surface3D('r(t,u)', r2, t_dom, u_dom)
EqPrint('r(t,u)', surf2.r)

In [ ]:
surf2.quickPlot()

In [ ]:
area, details = surf2.getArea()
display(
    details.jacobi,
    details.area
)

Vi ser en skive-agtig flade med hul i midten.

## Opgave 3
![](2022-12-16-15-04-14.png)

In [ ]:
#u, v = symbols('u_C v_C')

s3 = Matrix([u, v, sin(u)*cos(v)])
u_dom = (u, -pi, pi)
v_dom = (v, -pi, pi)
surf3 = Surface3D('s(u, v', s3, u_dom, v_dom)
EqPrint('s(u,v)', surf3.r)

In [ ]:
surf3.quickPlot()

![](2022-12-16-15-04-29.png)

In [ ]:
jac3 = surf3.getJacobi()
EqPrint('Jacobi_s', jac3)

![](2022-12-16-15-04-37.png)

In [ ]:
area, details = surf3.getArea()
display(
    *details
)

In [ ]:
simplify(area.evalf().subs({u:pi,v:pi}))

In [ ]:
print(details.jacobi)

Den kan ikke finde ud af integralet, så vi kan ikke finde arealet direkte.



Alternativt kan numerisk integration benyttes. Dette kan gøres ved hjælp af scipy.

In [ ]:
integrand = lambdify([u,v], details.jacobi.rhs)
result = scipy.integrate.dblquad(integrand, -pi, pi, -pi, pi)
print(result)

Her får vi et areal på $48.17968...$ med en tilhørende usikkerhed af meget lille størrelse. Vi kan sammenligne resultatet med hvis fladen var plan:

In [ ]:
EqPrint("(2\pi)^2",((2*pi)**2).evalf())

Det egentlige areal af fladen må nødvendigvis værre større end overstående, men kun en smule, hvorfor den numeriske udregning giver et resultat inden for det forventelige.

![](2022-12-16-15-04-47.png)

Indsætter $\pi/2$ på $v$'s plads og udregner Weingarten

In [ ]:
weingarten = surf3.getWeingarten()
display(weingarten)

weingarten = weingarten.subs({v: pi/2})
#display(weingarten)

Simplifier Weingarten...

In [ ]:
EqPrint('W(u,\pi/2)', simplify(weingarten))

![](2022-12-16-15-05-03.png)

Samme procedure som for Weingarten

In [ ]:
GaussK = surf3.getGaussK()
GaussK = GaussK.subs({v:pi/2})
display(GaussK)

Indsætter den konstante $v$

In [ ]:
GaussK = simplify(GaussK)
display(EqPrint('K(u,\pi/2)', GaussK))

![](2022-12-16-15-05-20.png)

Igen samme procedure

In [ ]:
MiddelH = surf3.getMiddelH()
display(EqPrint('H(u,v)', MiddelH))
MiddelH = MiddelH.subs({v:pi/2})
display(EqPrint('H(u, \pi/2)', MiddelH))

Vi observerer at $H(u) = 0$, når $v=\pi/2$

 Det omvendte vil også gælde  for $u$ med værdier der opfylder $sin(u)=0$, dvs. $-\pi$, $0$ og $\pi$.

~~At middel-krumningen er konstant 0, vil sige at fladen er mulig med et ustrækeligt stykke papir.~~

In [ ]:
u_dom = (u, -pi, pi)
v_dom = (v, -pi, pi)
surf3_2 = Surface3D('s_2(u, v', surf3.r.subs({v:pi/2}), u_dom, v_dom)
EqPrint('s_2(u,v)', surf3_2.r)

Vi observerer at parameterfremstillingen er gået fra flade til en ret linje efter at vi har sat $v=\pi/2$

![](2022-12-16-15-05-31.png)

In [ ]:
K3 = simplify(surf3.getGaussK().subs({v:pi/2}))
H3 = simplify(surf3.getMiddelH().subs({v:pi/2}))
display(K3, H3)

For at vi har et navlepunkt, skal det gælde at: $H^2 = K$ 

In [ ]:
eq3 = Eq(H3**2, K3)
eq3

Vi løser for overstående ligning

In [ ]:
sol3 = solve(eq3, dict=True)
display(
    sol3[0][u],
    sol3[1][u],
)

Der er altså tale om navlepunkter netop når $u=\pi/2$ eller $u=\pi -1/2$, når $v=\pi/2$

Slut...